In [321]:
# All the current hubs and how many people it takes care of
hubs = {"Upper Fraser Valley": 0, "South Vancouver": 0, "Burnaby": 0, "Collingwood": 0, "Abbotsford": 0, "Downtown Eastside": 0,
        "Surrey": 0, "Central Cariboo": 0, "North Pemberton": 0, "North Okanagan": 0, "Kimberley-Cranbrook": 0,
        "Lower Columbia": 0, "Strathcona": 0, "Tri-Cities": 0, "Delta": 0, "West End Vancouver": 0}
hubsCoordinate = {"Upper Fraser Valley": (49.17114, -121.95336), "South Vancouver": (49.22552, -123.0655), "Burnaby": (49.22538, -123.00434), 
                    "Collingwood": (49.23665, -123.03299), "Abbotsford": (49.0461, -122.2901), "Downtown Eastside": (49.28092, -123.0901),
                    "Surrey": (49.01806, -122.78841), "Central Cariboo": (52.13022, -122.13799), "North Pemberton": (50.32157, -122.81039),
                    "North Okanagan": (50.26701, -119.27201), "Kimberley-Cranbrook": (49.51449, -115.76031),
                    "Lower Columbia": (49.09714, -117.70748), "Strathcona": (50.0146, -125.24437),
                    "Tri-Cities": (49.2786, -122.79264), "Delta": (49.00834, -123.04659), "West End Vancouver": (49.2857, -123.13365)}

In [322]:
import io
import pandas as pd
import numpy as np

In [323]:
calls = pd.read_csv('/content/drive/MyDrive/BC211_CallsForFood2021(original & adjusted).csv')

In [324]:
!pip install haversine
import haversine as hs

In [325]:
def closestHub(cityCoordinate):
  # a function that returns the closest steward from a city and its distance
  loc1 = (float(cityCoordinate.split(',')[0]), float(cityCoordinate.split(',')[1]))
  distance = 0
  distanceList = []
  for hubName, loc2 in hubsCoordinate.items():
    # Use Haversine Distance
    distance = hs.haversine(loc1, loc2)
    distanceList.append((hubName, distance))
  distanceList.sort(key=lambda x:x[1])
  return distanceList[0]

In [326]:
def BOI(total, distance):
  # this function calculates the bail-out index of a certain area based on total calls and distance
  return total / distance

In [327]:
from pandas.core.arrays.numeric import T
bail_out_index = 0.15 # flexible
for i, r in calls.iterrows():
  # if the city is not in any hub and its BOI is bigger than n, assign that city to its closest hub
  if str(r['Food Hub']).isalnum() and BOI(r['Total'], closestHub(r['coordinate'])[1]) > bail_out_index:
    hubs[closestHub(r['coordinate'])[0]] += r['Total']
  # if the city is in an existing hub
  elif not str(r['Food Hub']).isalnum() and str(r['City']) != 'Vancouver':
    hubs[(r['hub'])] += r['Total']
  # Vancouver has 4 hubs, 5 stewards, divide the calls evenly
  elif str(r['City']) == 'Vancouver':
    hubs['South Vancouver'] += 156
    hubs["Collingwood"] += 156
    hubs['Downtown Eastside'] += 313
    hubs['West End Vancouver'] += 156

In [328]:
cover_rate = sum(hubs.values())/2644

0.9084720121028744

In [329]:
sorted_hubs = sorted(hubs.items(), key=lambda kv: kv[1])